In [10]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Lambda
import keras.applications.mobilenet_v2 as mobilenetv2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [11]:
# Load MobileNetV2 base model without the top layer
mobilenetv2_layer = mobilenetv2.MobileNetV2(include_top=False, 
                                            input_shape=(224, 224, 3),
                                            weights='../mobilenet-v2-keras-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5')

mobilenetv2_layer.trainable = False

In [12]:
# Create the model architecture
model = Sequential()
# Input layer
model.add(tf.keras.Input(shape=(224, 224, 3)))

# Lambda preprocessing layer
def mobilenetv2_preprocessing(img):
    return mobilenetv2.preprocess_input(img)

model.add(Lambda(mobilenetv2_preprocessing))

model.add(mobilenetv2_layer)

# Add the GlobalAveragePooling2D layer
model.add(GlobalAveragePooling2D())

# Update the Dense layer for 2 classes
model.add(Dense(2, activation='softmax'))

# Compile the model with a lower learning rate for fine-tuning
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['categorical_accuracy'])

# Print the updated model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_1 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [14]:
# train_dir = '../../trainingData/'
train_dir = 'trainingData/'

In [16]:
# Create an ImageDataGenerator with data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.1,
    rotation_range=15,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    # vertical_flip=True,
    validation_split=0.1
)

# Load training data with flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),      # Resizes images to 224x224
    batch_size=16,               # Number of images per batch
    class_mode='categorical', 
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

# Print class indices to verify the labels
print(f"Class Indices: {train_generator.class_indices}")

Found 324 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Class Indices: {'metal': 0, 'plastic': 1}


In [17]:
class_weights = {0: 1.0, 1: 1.0}

In [18]:
early_stop = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Stop after 5 epochs of no improvement
    restore_best_weights=True  # Restore the best weights
)

In [19]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [20]:
# Fine-tune the model on the new data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stop, reduce_lr],
    class_weight=class_weights,
    epochs=8
)

Epoch 1/8
20/20 [==============================] - 7s 254ms/step - loss: 1.0754 - categorical_accuracy: 0.4935 - val_loss: 0.8377 - val_categorical_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 2/8
20/20 [==============================] - 4s 221ms/step - loss: 0.7483 - categorical_accuracy: 0.4903 - val_loss: 0.6955 - val_categorical_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 3/8
20/20 [==============================] - 4s 214ms/step - loss: 0.6967 - categorical_accuracy: 0.4838 - val_loss: 0.6934 - val_categorical_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 4/8
20/20 [==============================] - 4s 212ms/step - loss: 0.6959 - categorical_accuracy: 0.4545 - val_loss: 0.6960 - val_categorical_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 5/8
20/20 [==============================] - 4s 210ms/step - loss: 0.6936 - categorical_accuracy: 0.5065 - val_loss: 0.6915 - val_categorical_accuracy: 0.5938 - lr: 1.0000e-04
Epoch 6/8
20/20 [==============================] - 4s 212ms/step - loss: 0.6921 - categori

In [65]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# Save the fine-tuned model
model.save('fine_tuned_plastic_metal_model.h5')

13/13 [==============================] - 2s 140ms/step - loss: 0.1751 - categorical_accuracy: 0.9798
Validation Loss: 0.17507001757621765, Validation Accuracy: 0.9797979593276978


### Inference

In [21]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the fine-tuned model
model = load_model('fine_tuned_plastic_metal_model.h5')

In [28]:
# img_path = '../sample_data/metal_coke.jpg'
# img_path = '../sample_data/plastic_bottle_partial.jpg'
# img_path = 'trainingData/metal/image_20240922_040431_4.jpg'
# img_path = 'trainingData/metal/image_20240922_034123_3.jpg'
# img_path = 'trainingData/plastic/image_20240922_023123_10.jpg'
img_path = 'trainingData/plastic/image_20240922_031110_3.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = mobilenetv2.preprocess_input(img_array)

In [29]:
# Predict using the fine-tuned model
predictions = model.predict(img_array)
print(predictions)
class_index = np.argmax(predictions, axis=1)

# Map the predicted index to class label
class_labels = ('metal', 'plastic')
predicted_class = class_labels[class_index[0]]
print(f'Predicted Class: {predicted_class}')

1/1 [==============================] - 0s 29ms/step
[[0.75754875 0.24245124]]
Predicted Class: metal
